In [27]:
import matplotlib.pyplot as plt
import copy, numpy as np

def sigmoid(x):
    return (1/(1+np.exp(-x)))

def sigmoid_deriv(x):
    return (x*(1-x));


In [34]:
int2bin = {}
binary_dim = 8

largest_num = pow(2,binary_dim)
binary = np.unpackbits(
    np.array([range(largest_num)],dtype=np.uint8).T,axis=1)

for i in range(largest_num):
    int2bin[i] = binary[i]
    
# input var
alpha = .1
input_dim = 2
hidden_dim = 16
output_dim = 1

# matrices - initialize between -1 to 1
synapse_0 = 2*np.random.random((input_dim,hidden_dim))-1;
synapse_1 = 2*np.random.random((hidden_dim,output_dim))-1;
synapse_h = 2*np.random.random((hidden_dim,hidden_dim))-1; #recurr

syn_0_update = np.zeros_like(synapse_0);
syn_1_update = np.zeros_like(synapse_1);
syn_h_update = np.zeros_like(synapse_h);


# train
itr = 100000
for j in range(itr):
    
    # gen some random addition problem
    a_int = np.random.randint(largest_num/2);
    a = int2bin[a_int];
    
    b_int = np.random.randint(largest_num/2);
    b = int2bin[b_int];
    
    # ground truth
    c_int = a_int + b_int;
    c = int2bin[c_int];
    
    # guess
    d = np.zeros_like(c)
    
    overallErr = 0;
    
    layer_2_derivs = list();
    layer_1_values = list();
    layer_1_values.append(np.zeros(hidden_dim));
    #print(a_int,b_int)
    #print(a,b)
    for position in range(binary_dim):
        X = np.array([[a[binary_dim - position - 1],b[binary_dim - position - 1]]])
        #print(X)
        y = np.array([[c[binary_dim - position -1]]]).T
        
        # hidden layer (input ~+ prev_hidden)
        
        layer_1 = sigmoid(np.dot(X,synapse_0) + np.dot(layer_1_values[-1],synapse_h))
        #print(layer_1_values)
        layer_2 = sigmoid(np.dot(layer_1,synapse_1))
        
        # lost
        layer_2_err = y-layer_2
        layer_2_derivs.append((layer_2_err)*sigmoid_deriv(layer_2))
        overallErr += np.abs(layer_2_err[0])
        
        # decode estimate
        d[binary_dim - position - 1] = np.round(layer_2[0][0])
        
        # store hidden layer for next timestep
        layer_1_values.append(copy.deepcopy(layer_1))
        
    future_layer_1_deriv = np.zeros(hidden_dim)
    
    for position in range(binary_dim):

        X = np.array([[a[position],b[position]]])
        layer_1 = layer_1_values[-position-1]
        prev_layer_1 = layer_1_values[-position-2]

        # error at output
        layer_2_deriv = layer_2_derivs[-position-1]

        layer_1_deriv = (future_layer_1_deriv.dot(synapse_h.T) + layer_2_deriv.dot(synapse_1.T))*sigmoid_deriv(layer_1)

        # update
        syn_1_update += np.atleast_2d(layer_1).T.dot(layer_2_deriv)
        syn_h_update += np.atleast_2d(prev_layer_1).T.dot(layer_2_deriv)
        syn_0_update += X.T.dot(layer_1_deriv)

        future_layer_1_delta = layer_1_deriv
        
    synapse_0 += syn_0_update * alpha;
    synapse_1 += syn_1_update * alpha;
    synapse_h += syn_h_update * alpha;    
    
    syn_0_update = 0
    syn_1_update = 0
    syn_h_update = 0
    
    if(j % 1000 == 0):
        print("Error: %s", overallErr );
        print("Pred: %s", d)
        print("Truth: %s", c)
        out = 0;
        
        for index,x in enumerate(reversed(d)):
            out += x*pow(2,index)
        
        print("%s",a_int," + %s",b_int,"= %s",c)
        print("--------------")
    

Error: %s [ 4.04898322]
Pred: %s [1 1 1 1 1 1 1 1]
Truth: %s [1 1 1 0 0 0 0 1]
%s 102  + %s 123 = %s [1 1 1 0 0 0 0 1]
--------------
Error: %s [ 4.06219589]
Pred: %s [0 0 0 0 0 0 0 0]
Truth: %s [1 0 0 1 1 1 0 1]
%s 123  + %s 34 = %s [1 0 0 1 1 1 0 1]
--------------
Error: %s [ 4.04555422]
Pred: %s [1 1 1 1 1 0 1 1]
Truth: %s [1 0 1 0 0 1 0 1]
%s 69  + %s 96 = %s [1 0 1 0 0 1 0 1]
--------------
Error: %s [ 3.4807307]
Pred: %s [1 0 0 0 1 0 1 0]
Truth: %s [1 0 0 0 1 0 1 0]
%s 111  + %s 27 = %s [1 0 0 0 1 0 1 0]
--------------
Error: %s [ 4.28098564]
Pred: %s [0 1 0 1 1 0 1 1]
Truth: %s [1 0 1 0 0 0 1 1]
%s 40  + %s 123 = %s [1 0 1 0 0 0 1 1]
--------------
Error: %s [ 3.71642765]
Pred: %s [0 1 1 0 1 1 0 0]
Truth: %s [0 1 1 1 0 1 0 0]
%s 109  + %s 7 = %s [0 1 1 1 0 1 0 0]
--------------
Error: %s [ 4.25398981]
Pred: %s [0 0 1 1 1 1 0 1]
Truth: %s [1 1 0 0 1 0 1 1]
%s 95  + %s 108 = %s [1 1 0 0 1 0 1 1]
--------------
Error: %s [ 3.64013337]
Pred: %s [0 0 0 0 1 0 0 0]
Truth: %s [1 0 0 0 1

Error: %s [ 4.01162149]
Pred: %s [0 0 0 0 0 0 0 0]
Truth: %s [0 0 1 1 1 0 1 0]
%s 12  + %s 46 = %s [0 0 1 1 1 0 1 0]
--------------
Error: %s [ 4.09575561]
Pred: %s [0 0 0 0 0 0 0 0]
Truth: %s [1 1 0 0 1 0 1 0]
%s 108  + %s 94 = %s [1 1 0 0 1 0 1 0]
--------------
Error: %s [ 3.56506349]
Pred: %s [1 1 1 1 1 1 1 1]
Truth: %s [0 0 1 0 1 0 1 1]
%s 34  + %s 9 = %s [0 0 1 0 1 0 1 1]
--------------
Error: %s [ 3.70281286]
Pred: %s [1 1 1 1 1 1 1 1]
Truth: %s [0 0 1 0 1 0 0 1]
%s 1  + %s 40 = %s [0 0 1 0 1 0 0 1]
--------------
Error: %s [ 3.10842287]
Pred: %s [0 0 0 0 0 0 0 1]
Truth: %s [0 1 0 0 1 0 0 1]
%s 57  + %s 16 = %s [0 1 0 0 1 0 0 1]
--------------
Error: %s [ 3.47785268]
Pred: %s [0 0 0 0 0 0 0 1]
Truth: %s [0 1 1 1 0 0 0 1]
%s 87  + %s 26 = %s [0 1 1 1 0 0 0 1]
--------------
Error: %s [ 3.50364011]
Pred: %s [1 1 1 1 1 1 1 1]
Truth: %s [0 1 1 0 1 0 1 1]
%s 73  + %s 34 = %s [0 1 1 0 1 0 1 1]
--------------
Error: %s [ 4.10442387]
Pred: %s [0 0 0 0 0 0 0 0]
Truth: %s [1 0 1 1 0 1 1 0

In [ ]:
x = np.zeros(21)
y = np.zeros(21)
for i in range(-10,11):
    x[i+10] = sigmoid(i);

for i in range(-10,11):
    y[i+10] = sigmoid_derive(x[i+10]);
plt.plot(x)
plt.plot(y)
plt.show()